In [8]:
import boto3
import pandas as pd
import time 
import os

In [9]:
client = boto3.client('comprehend',
                      aws_access_key_id= os.getenv("aws_access_key_id"),
                      aws_secret_access_key= os.getenv("aws_secret_access_key"),
                      region_name='us-east-1')

In [10]:
def analyze_sentiment(text):
    try:
        response = client.detect_sentiment(Text=text, LanguageCode='en')
        # Extrae tanto el sentimiento como los puntajes de confianza
        sentiment = response['Sentiment']
        sentiment_scores = response['SentimentScore']
        return sentiment, sentiment_scores
    except Exception as e:
        print(f"Error al analizar el sentimiento: {e}")
        return None, None

In [11]:
def analyze_targeted_sentiment(text):
    try:
        #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/comprehend/client/detect_targeted_sentiment.html
        response = client.detect_targeted_sentiment(Text=text, LanguageCode='en')
        entities = response['Entities']
        sentiment_results = []
        for entity in entities:
            for mention in entity['Mentions']:
                sentiment = mention['MentionSentiment']['Sentiment']
                text = mention['Text']
                sentiment_result = {
                    'Sentiment': sentiment,
                    'Text': text
                }
                sentiment_results.append(sentiment_result)
        return sentiment_results
    except Exception as e:
        print(f"Error al analizar el sentimiento: {e}")
        return None

In [12]:
df=pd.read_csv("data/data_raw_full_webster.csv")

In [13]:
# Aplica la función a la columna 'Review' y extrae los resultados en nuevas columnas
df[['Sentiment_gen', 'SentimentScores_gen']] = df['Review'].apply(
    lambda x: pd.Series(analyze_sentiment(x))
)

# Si quieres descomponer los puntajes de sentimiento en columnas separadas
#df[['Positive_gen', 'Negative_gen', 'Neutral_gen', 'Mixed_gen']] = df['SentimentScores_gen'].apply(pd.Series)

# Remover la columna de puntajes agregados si ya no es necesaria
#df.drop('SentimentScores_gen', axis=1, inplace=True)


Error al analizar el sentimiento: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Error al analizar el sentimiento: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Error al analizar el sentimiento: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Error al analizar el sentimiento: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Error al analizar el sentimiento: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Error al analizar el sentimiento: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Error al analizar el sentimiento: Parameter validati

In [18]:
df[['Positive_gen', 'Negative_gen', 'Neutral_gen', 'Mixed_gen']] = df['SentimentScores_gen'].apply(pd.Series)


In [21]:
df.drop('SentimentScores_gen', axis=1, inplace=True)


In [14]:
def entity_type_count(response):
    entity_count = {}
    for entity in response['Entities']:
        entity_type = entity['Type']
        if entity_type not in entity_count:
            entity_count[entity_type] = 0
        entity_count[entity_type] += 1
    return entity_count

# Aplica esta función según sea necesario para extraer datos específicos

In [15]:
def sentiment_majority(text):
    sentiment_counts = {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0, 'MIXED': 0}
    try:
        response = client.detect_targeted_sentiment(Text=text, LanguageCode='en')
        for entity in response['Entities']:
            # Comprueba si 'Sentiment' está en el diccionario de la entidad
            if 'Sentiment' in entity:
                sentiment_counts[entity['Sentiment']] += 1
    except Exception as e:
        print(f"Se produjo un error: {e}")
    majority_sentiment = max(sentiment_counts, key=sentiment_counts.get)
    return majority_sentiment

df['MajoritySentiment'] = df['Review'].apply(sentiment_majority)


Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types:

In [ ]:
def sentiment_count_by_type(text):
    sentiment_type_count = {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0, 'MIXED': 0}
    try:
        response = client.detect_targeted_sentiment(Text=text, LanguageCode='en')
        for entity in response['Entities']:
            # Comprueba si 'Sentiment' está en el diccionario de la entidad
            if 'Sentiment' in entity:
                sentiment_type_count[entity['Sentiment']] += 1
    except Exception as e:
        print(f"Se produjo un error: {e}")
    return sentiment_type_count

# Agrega esto como varias columnas en el DataFrame
for sentiment in ['POSITIVE', 'NEGATIVE', 'NEUTRAL', 'MIXED']:
    df[f'{sentiment}_Count'] = df['Review'].apply(lambda x: sentiment_count_by_type(x)[sentiment])


Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types: <class 'str'>
Se produjo un error: Parameter validation failed:
Invalid type for parameter Text, value: nan, type: <class 'float'>, valid types:

In [ ]:
def weighted_sentiment_score(text):
    scores = {'POSITIVE': 0, 'NEGATIVE': 0, 'NEUTRAL': 0, 'MIXED': 0}
    total_scores = {'POSITIVE': 0.01, 'NEGATIVE': 0.01, 'NEUTRAL': 0.01, 'MIXED': 0.01}  # Evitar división por cero
    try:
        response = client.detect_targeted_sentiment(Text=text, LanguageCode='en')
        for entity in response['Entities']:
            # Comprueba si 'Sentiment' y 'SentimentScore' están en el diccionario de la entidad
            if 'Sentiment' in entity and 'SentimentScore' in entity:
                sentiment = entity['Sentiment']
                score = entity['SentimentScore'][sentiment]
                scores[sentiment] += score
                total_scores[sentiment] += 1
    except Exception as e:
        print(f"Se produjo un error: {e}")
    for key in scores:
        scores[key] /= total_scores[key]
    return scores

# Agrega esto también como varias columnas en el DataFrame
for sentiment in ['POSITIVE', 'NEGATIVE', 'NEUTRAL', 'MIXED']:
    df[f'{sentiment}_Score'] = df['Review'].apply(lambda x: weighted_sentiment_score(x)[sentiment])



In [22]:
df.head()

,Review,DS,Bank Name,data_source,Bank ID,state,Sentiment_gen,MajoritySentiment,POSITIVE_Count,Positive_gen,Negative_gen,Neutral_gen,Mixed_gen
0,Mere words cannot express the deep felt gratit...,04/18/2024 19:32:14,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.870884,0.027722,0.098281,0.003113
1,i came across Mrs. Jennifer when she appeared ...,03/22/2024 16:38:36,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.672443,0.000645,0.326820,0.000092
2,"Excellent Service""! Accountant is kind and ver...",01/07/2024 21:34:14,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.990623,0.000058,0.009292,0.000026
3,Good brokerage and trading platform. I love th...,10/29/2023 20:36:58,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.992917,0.000059,0.006926,0.000098
4,REGINA NATHAN is a TOP NOTCH investing compan...,09/20/2023 19:17:29,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.947814,0.000749,0.051270,0.000167


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15967 entries, 0 to 15966
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Review               11571 non-null  object 
 1   DS                   15776 non-null  object 
 2   Bank Name            15967 non-null  object 
 3   data_source          15967 non-null  object 
 4   Bank ID              15967 non-null  int64  
 5   state                15967 non-null  object 
 6   Sentiment_gen        11562 non-null  object 
 7   SentimentScores_gen  11562 non-null  object 
 8   MajoritySentiment    15967 non-null  object 
 9   POSITIVE_Count       15967 non-null  int64  
 10  Positive_gen         11562 non-null  float64
 11  Negative_gen         11562 non-null  float64
 12  Neutral_gen          11562 non-null  float64
 13  Mixed_gen            11562 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 1.7+ MB


In [23]:
df.describe()

,Bank ID,POSITIVE_Count,Positive_gen,Negative_gen,Neutral_gen,Mixed_gen
count,15967.000000,15967.0,11562.000000,11562.000000,11562.000000,11562.000000
mean,18.433770,0.0,0.456594,0.432796,0.065239,0.045370
std,7.927122,0.0,0.471243,0.458704,0.175235,0.156247
min,1.000000,0.0,0.000037,0.000016,0.000022,0.000002
25%,21.000000,0.0,0.001816,0.000101,0.000659,0.000039
50%,22.000000,0.0,0.124417,0.106149,0.003528,0.000168
75%,23.000000,0.0,0.996834,0.982301,0.027302,0.003992
max,24.000000,0.0,0.999856,0.999831,0.999650,0.999568


In [24]:
df.drop(columns=["POSITIVE_Count","NEGATIVE_Count","NEUTRAL_Count","MIXED_Count","POSITIVE_Score","NEGATIVE_Score","NEUTRAL_Score","MIXED_Score"],inplace=True)

KeyError: "['NEGATIVE_Count', 'NEUTRAL_Count', 'MIXED_Count', 'POSITIVE_Score', 'NEGATIVE_Score', 'NEUTRAL_Score', 'MIXED_Score'] not found in axis"

In [26]:
#data=pd.read_csv('data/data_processed_webster.csv')
df['Sentiment_Score'] = df['Positive_gen'] * 1 + df['Neutral_gen'] * 0.5 + df['Negative_gen'] * 0
df


,Review,DS,Bank Name,data_source,Bank ID,state,Sentiment_gen,MajoritySentiment,POSITIVE_Count,Positive_gen,Negative_gen,Neutral_gen,Mixed_gen,Sentiment_Score
0,Mere words cannot express the deep felt gratit...,04/18/2024 19:32:14,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.870884,0.027722,0.098281,0.003113,0.920025
1,i came across Mrs. Jennifer when she appeared ...,03/22/2024 16:38:36,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.672443,0.000645,0.326820,0.000092,0.835853
2,"Excellent Service""! Accountant is kind and ver...",01/07/2024 21:34:14,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.990623,0.000058,0.009292,0.000026,0.995270
3,Good brokerage and trading platform. I love th...,10/29/2023 20:36:58,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.992917,0.000059,0.006926,0.000098,0.996380
4,REGINA NATHAN is a TOP NOTCH investing compan...,09/20/2023 19:17:29,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,POSITIVE,0,0.947814,0.000749,0.051270,0.000167,0.973449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15962,"Webster Financial Corporation (NYSE:WBS), the ...",2024-04-24T20:43:08Z,Webster Bank,news,24,EEUU,NEUTRAL,POSITIVE,0,0.003693,0.008020,0.988272,0.000016,0.497829
15963,"April 24, 2024 at 16:41 PM EDT Webster Financ...",2024-04-24T20:42:11Z,Webster Bank,news,24,EEUU,NEUTRAL,POSITIVE,0,0.005492,0.005953,0.988536,0.000018,0.499761
15964,(RTTNews) - Webster Financial Corporation (WBS...,2024-04-24T08:42:56Z,Webster Bank,news,24,EEUU,NEUTRAL,POSITIVE,0,0.001876,0.012448,0.985664,0.000012,0.494708
15965,"STAMFORD, Conn.–(BUSINESS WIRE)–Webster Financ...",2024-04-24T08:27:39Z,Webster Bank,news,24,EEUU,None,POSITIVE,0,NaN,NaN,NaN,NaN,NaN


In [27]:
# Intentar convertir la columna 'DS' a formato de fecha y hora
df['DS_converted'] = pd.to_datetime(df['DS'], errors='coerce')

# Formatear las fechas para incluir solo año, mes y día
df['DS_formatted'] = df['DS_converted'].dt.strftime('%Y-%m-%d')

# Identificar las filas donde la conversión inicial falló
failed_conversions = df[df['DS_converted'].isna()]

# Si deseas, puedes exportar las conversiones fallidas para analizarlas manualmente
failed_conversions.to_csv('fechas_fallidas.csv', index=False)

# Reemplazar las fechas fallidas con un método específico si es necesario
# Aquí aplicamos conversiones específicas para formatos identificados manualmente
for format in ['%d-%m-%Y %H:%M:%S', '%Y-%m-%dT%H:%M:%SZ']:
    mask = df['DS_converted'].isna()
    df.loc[mask, 'DS_converted'] = pd.to_datetime(df.loc[mask, 'DS'], format=format, errors='coerce')
    df.loc[mask, 'DS_formatted'] = df.loc[mask, 'DS_converted'].dt.strftime('%Y-%m-%d')

# Verificar cuántas aún fallan
still_failing = df[df['DS_converted'].isna()]




In [28]:
del df['DS']
del df['DS_converted']
del df['MajoritySentiment']
df.rename(columns={'DS_formatted':'ds'},inplace=True)
df

,Review,Bank Name,data_source,Bank ID,state,Sentiment_gen,POSITIVE_Count,Positive_gen,Negative_gen,Neutral_gen,Mixed_gen,Sentiment_Score,ds
0,Mere words cannot express the deep felt gratit...,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,0,0.870884,0.027722,0.098281,0.003113,0.920025,2024-04-18
1,i came across Mrs. Jennifer when she appeared ...,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,0,0.672443,0.000645,0.326820,0.000092,0.835853,2024-03-22
2,"Excellent Service""! Accountant is kind and ver...",Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,0,0.990623,0.000058,0.009292,0.000026,0.995270,2024-01-07
3,Good brokerage and trading platform. I love th...,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,0,0.992917,0.000059,0.006926,0.000098,0.996380,2023-10-29
4,REGINA NATHAN is a TOP NOTCH investing compan...,Chelsea Gronto Bank,facebook,2,EEUU,POSITIVE,0,0.947814,0.000749,0.051270,0.000167,0.973449,2023-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15962,"Webster Financial Corporation (NYSE:WBS), the ...",Webster Bank,news,24,EEUU,NEUTRAL,0,0.003693,0.008020,0.988272,0.000016,0.497829,2024-04-24
15963,"April 24, 2024 at 16:41 PM EDT Webster Financ...",Webster Bank,news,24,EEUU,NEUTRAL,0,0.005492,0.005953,0.988536,0.000018,0.499761,2024-04-24
15964,(RTTNews) - Webster Financial Corporation (WBS...,Webster Bank,news,24,EEUU,NEUTRAL,0,0.001876,0.012448,0.985664,0.000012,0.494708,2024-04-24
15965,"STAMFORD, Conn.–(BUSINESS WIRE)–Webster Financ...",Webster Bank,news,24,EEUU,None,0,NaN,NaN,NaN,NaN,NaN,2024-04-24


In [29]:
df.to_csv('data/data_processed_webster.csv',index=False)